In [49]:
import pandas as pd
import numpy as np

na_values = ['<Null>', '']
df_1 = pd.read_csv(r'C:\Users\55199\OneDrive - Suzano S A\Área de Trabalho\OneDrive Suzano\OneDrive - Suzano S A\Documentos\SAP BW ECC\Datasphere\ADFIFT006.csv', sep='\t', dtype=str, na_values=na_values)
df_2 = pd.read_csv(r'C:\Users\55199\OneDrive - Suzano S A\Área de Trabalho\OneDrive Suzano\OneDrive - Suzano S A\Documentos\SAP BW ECC\Datasphere\ADFIFT005.csv', sep=';', dtype=str, na_values=na_values)

In [71]:
df_2.columns

Index(['0AC_DOC_NO', '0AC_LEDGER', '0AC_LEDGER___T', '0BUS_AREA',
       '0BUS_AREA___T', '0CHRT_ACCTS', '0CHRT_ACCTS___T', '0COMP_CODE',
       '0COMP_CODE___T', '0COORDER', '0COORDER___T', '0COORD_TYPE',
       '0COORD_TYPE___T', '0COSTCENTER', '0COSTCENTER___T', '0CO_AREA',
       '0CO_AREA___T', '0CURRENCY', '0CURRENCY___T', '0CURTYPE',
       '0CURTYPE___T', '0DIVISION', '0DIVISION___T', '0DOC_TYPE',
       '0DOC_TYPE___T', '0FISCPER', '0FISCVARNT', '0FISCYEAR', '0FI_AWTYP',
       '0FI_AWTYP___T', '0FMD_FSIAC5', '0FUNC_AREA', '0FUNC_AREA___T',
       '0GL_ACCOUNT', '0GL_ACCOUNT___T', '0ID_TXT50', '0ID_TXT50___T',
       '0ITEM_NUM', '0LOC_CURRC2', '0LOC_CURRC2___T', '0MOVE_TYPE',
       '0MOVE_TYPE___T', '0PCOMPANY', '0PCOMPANY___T', '0PLANT', '0PLANT___T',
       '0PROFIT_CTR', '0PROFIT_CTR___T', '0REF_DOC_NO', '0REQTSN',
       '0RESP_CCTR', '0RESP_CCTR___T', '0SALES_DIST', '0SALES_DIST___T',
       '0UNIT', '0UNIT___T', '0VERSION', '0VERSION___T', '0VTYPE',
       '0VTYPE___T'

In [86]:
def filter_df(df):
    def filter_sga(row):
        if row['0COORD_TYPE'] == 'EMB':
            return 'X'
        else:
            gl_exception = ['0004515100', '0004111015']
            if row["0COORD_TYPE"] == '' and row["ZCA_394"] == 'O' and row['0GL_ACCOUNT'] not in gl_exception:
                return 'X'
            elif row["0COSTCENTER"] == '' and (row['0COMP_CODE'] != 'PCEL' and row['0GL_ACCOUNT'] != '0004422130'):
                return 'X'
            else:
                return ''
                    
                    
    df = df.fillna('')
    df['GL_INT'] = df['0GL_ACCOUNT'].apply(lambda x: int(x) if x != '' else np.nan)
    df['FUNC_TYPE'] = df['0FUNC_AREA'].apply(lambda x: x[:3])
    
    gl_to_exclude = ['0004422755','0004511001','0004511090','0004512001','0004512040','0004512060','0004512070','0004512090','0004513001',
                        '0004513030','0004515001','0004515005','0004515010','0004515040','0004515050','0004515053','0004514020','0004514040',
                        '0004514070','0004514100','0004515054','0004513031','0004515052','0004515056','0004516002','0004514060','0004514080',
                        '0004515058','0004514030']
    
    is_auak = (df['0FI_AWTYP'] == 'AUAK')
    gl_in_range = (df['GL_INT'] < 4000000) | (df['GL_INT'] >= 5000000)
    gl_in_list = df['0GL_ACCOUNT'].apply(lambda x: x in gl_to_exclude or x == '')
    
    gl_mask = is_auak & (gl_in_range | gl_in_list)
    
    func_to_exclude = ['INV', 'PRO']
    func_mask = df['FUNC_TYPE'].apply(lambda x: x in func_to_exclude)
    mask = gl_mask|func_mask
    
    df['EXCLUIR'] = ''
    df.loc[mask, 'EXCLUIR'] = 'X'
    
    is_cspc = (df['0COMP_CODE']=='CSPC')
    not_marked = (df['EXCLUIR']!='X')
    
    mask = not_marked&is_auak&is_cspc
    df.loc[mask, 'EXCLUIR'] = df.loc[mask].apply(filter_sga, axis=1)
    
    return df.drop(columns=['GL_INT', 'FUNC_TYPE'])


df_output = filter_df(df_2)

In [90]:
df_output.loc[df_output['EXCLUIR'] == 'X']

,0AC_DOC_NO,0AC_LEDGER,0AC_LEDGER___T,0BUS_AREA,0BUS_AREA___T,0CHRT_ACCTS,0CHRT_ACCTS___T,0COMP_CODE,0COMP_CODE___T,0COORDER,...,ZCA_394,ZCA_394___T,0BALANCE,0CREDIT,0DEBIT,0DEB_CRE_L2,0QUANTITY,ZKF_1769,1ROWCOUNT,EXCLUIR
10,1064460995,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,TLS-CCSSZ,...,O,CustoComercialização,-2279.20 BRL,-2279.20 BRL,0.00 BRL,0.00,0.000,-2279.20 BRL,1,X
11,1063129921,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,LIS-PCGUA,...,O,CustoComercialização,139.10 BRL,139.10 BRL,0.00 BRL,0.00,0.000,139.10 BRL,1,X
43,1062780628,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,,...,O,CustoComercialização,272.25 BRL,0.00 BRL,272.25 BRL,0.00,0.000,272.25 BRL,1,X
52,1063912188,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,,...,O,CustoComercialização,3086.70 BRL,0.00 BRL,3086.70 BRL,0.00,0.000,3086.70 BRL,1,X
65,1062780554,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,,...,O,CustoComercialização,116478.75 BRL,0.00 BRL,116478.75 BRL,0.00,0.000,116478.75 BRL,1,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413611,1063823027,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,SZB-CCBTP,...,O,CustoComercialização,97.20 BRL,97.20 BRL,0.00 BRL,0.00,0.000,97.20 BRL,1,X
413617,1064437331,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,LIS-CCPER,...,O,CustoComercialização,-1424.50 BRL,-1424.50 BRL,0.00 BRL,0.00,0.000,-1424.50 BRL,1,X
413655,1062912200,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,,...,O,CustoComercialização,187256.62 BRL,0.00 BRL,187256.62 BRL,0.00,0.000,187256.62 BRL,1,X
413668,1063822597,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,,...,O,CustoComercialização,5276.50 BRL,0.00 BRL,5276.50 BRL,0.00,0.000,5276.50 BRL,1,X


In [80]:
df_output

,0AC_DOC_NO,0AC_LEDGER,0AC_LEDGER___T,0BUS_AREA,0BUS_AREA___T,0CHRT_ACCTS,0CHRT_ACCTS___T,0COMP_CODE,0COMP_CODE___T,0COORDER,...,0BALANCE,0CREDIT,0DEBIT,0DEB_CRE_L2,0QUANTITY,ZKF_1769,1ROWCOUNT,GL_INT,FUNC_TYPE,EXCLUIR
0,1063378248,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,MUB-CBPCL,...,1010.04 BRL,0.00 BRL,1010.04 BRL,0.00,0.000,1010.04 BRL,1,4111025,CCM,
1,1062920613,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,SZS-PCMEX,...,181.50 BRL,0.00 BRL,181.50 BRL,0.00,0.000 TON,181.50 BRL,1,4513001,CCM,
2,1063552683,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,PA1004-RODOV,...,-6.62 BRL,-6.62 BRL,0.00 BRL,0.00,0.000 TON,-6.62 BRL,1,4441050,CML,
3,1063934155,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,SZD-CBVDA,...,-3.77 BRL,-3.77 BRL,0.00 BRL,0.00,0.000 TON,-3.77 BRL,1,4441050,CML,
4,1063062942,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,PA1011-RODOV,...,352.34 BRL,0.00 BRL,352.34 BRL,0.00,0.000 TON,352.34 BRL,1,4441050,CML,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413668,1063822597,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,,...,5276.50 BRL,0.00 BRL,5276.50 BRL,0.00,0.000,5276.50 BRL,1,4512060,CCM,X
413669,1062377725,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,,...,84.06 BRL,0.00 BRL,84.06 BRL,0.00,0.000 TON,84.06 BRL,1,4422012,ADM,
413670,1064116793,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,SZ-MIPAP,...,-38.57 BRL,-38.57 BRL,0.00 BRL,0.00,0.000 TON,-38.57 BRL,1,4441051,CML,
413671,1063914304,FT,,,,PCCO,Plano de Contas Corporativo,CSPC,SUZANO S.A.,,...,1113.37 BRL,0.00 BRL,1113.37 BRL,0.00,0.000,1113.37 BRL,1,4515001,CCM,X


In [50]:
key_cols_006 = ['FISCPER', 'AC_DOC_NO', 'CURTYPE', 'FISCVARNT', 'COMP_CODE', 'GL_ACCOUNT',
            'CHRT_ACCTS', 'CO_AREA', 'COORDER', 'VERSION', 'VTYPE']
key_cols_005 = ['0'+col for col in key_cols_006]


In [51]:
mask = (df_2['0COORDER'].apply(lambda x: not pd.isna(x)))&(df_2['0COORD_TYPE'] != 'DIVS')
df_005 = df_2.loc[mask].drop(columns='0RESP_CCTR')

In [52]:
df_006 = df_1.rename(columns={'COSTCENTER':'RESP_CCTR'})

In [53]:
key_combo_005 = df_005.drop_duplicates(subset=key_cols_005)[key_cols_005].values
key_combo_006 = df_006.drop_duplicates(subset=key_cols_006+['RESP_CCTR'])[key_cols_006].values

In [63]:
for i, row in df_006.iterrows():
    row.fillna('')[key_cols_006[0]]
    break

In [65]:
int('004')

4

In [66]:
np.nan > 0

False

In [59]:
pd.DataFrame(columns=key_cols_006, data=key_combo_006)

,FISCPER,AC_DOC_NO,CURTYPE,FISCVARNT,COMP_CODE,GL_ACCOUNT,CHRT_ACCTS,CO_AREA,COORDER,VERSION,VTYPE
0,2022012,1064003492,00,K4,CSPC,0004221001,PCCO,NaN,NaN,001,010
1,2022012,1064003492,00,K4,CSPC,0004221001,PCCO,NaN,NaN,001,010
2,2022012,1064003492,00,K4,CSPC,0004221001,PCCO,NaN,NaN,001,010
3,2022012,1064003492,00,K4,CSPC,0004221001,PCCO,NaN,NaN,001,010
4,2022012,1064003492,00,K4,CSPC,0004221001,PCCO,NaN,NaN,001,010
...,...,...,...,...,...,...,...,...,...,...,...
1061647,2022012,1062539186,10,K4,CSPC,0004441010,PCCO,NaN,000002004933,001,010
1061648,2022012,1062539185,50,K4,CSPC,0004441010,PCCO,NaN,000002004929,001,010
1061649,2022012,1062539186,10,K4,CSPC,0004441010,PCCO,NaN,000002004930,001,010
1061650,2022012,1062539186,10,K4,CSPC,0004441010,PCCO,NaN,000002004929,001,010


In [56]:
shared_row = []
for row in key_cols_005:
    if row in key_cols_006:
        shared_row.append(row)


In [57]:
shared_row

[]

In [28]:
df_resp_cctr = df_merge.loc[~df_merge['RESP_CCTR'].apply(pd.isna)]

In [29]:
df_resp_cctr

,0AC_DOC_NO,0AC_LEDGER,0AC_LEDGER___T,0BUS_AREA,0BUS_AREA___T,0CHRT_ACCTS,0CHRT_ACCTS___T,0COMP_CODE,0COMP_CODE___T,0COORDER,...,COORDER,RESP_CCTR,CO_AREA,CURTYPE,FISCPER,FISCVARNT,GL_ACCOUNT,CHRT_ACCTS,VERSION,VTYPE
